<img style="float: right;"  src="images/LogoP.jpg" width="200">

# AC Module Reference

This a Jupyter Notebook Reference document for the SLab projects

Version 1.0 (7/3/2019) License information is at the end of the document

---

This **AC Module Reference Notebook** describes the **AC Module** commands.  
Those are the commands contained in the **ac.py** source file.


## Import and board connection

In order to use the AC commands, we first need to import the AC module. This is an interactive módule, so, in order to execute the code examples, you should import it executing the cell below.

We will also import the main **SLab** module as it includes some needed commands.

In [ ]:
# Import the main SLab module
import slab

# Import the AC module
import slab.ac as ac

If you want to try commands that interact with the **hardware board** using the examples in this document, you will need to connect to the board. You can use the following code cell for that. Note, however, that if you are using this document as reference while working on another SLab document, you can only have one connection, at the same time, with the board.

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

When you finish interacting with the board you can disconnect from it using the following code.

In [ ]:
# Disconnect from the board
slab.disconnect()

# Command Index

The AC SLab module contains several commands that ease the AC measurements. The following list contains all the commands in the AC module. You can use the hiperlinks to jump to one of the commands.

###  Frequency Response Commands

[sineGain](#sineGain) : Measures gain at one ADC for a sinewave tone   
[sineGainAll](#sineGainAll) : Measures gain at all ADCs for a sinewave tone  
[freqResponse](#freqResponse) : Measures gain for one ADC and several frequencies    
[freqResponseAll](#freqResponseAll) : Measures gain for all ADCs and several frequencies    

###  Frequency Plot Commands   

[plotBode](#plotBode) : Draws a Bode plot  
[plotFreq](#plotFreq) : Draws a linear frequency response plot    
[bodeResponse](#bodeResponse) : Measures and draws a Bode plot    

###  Utility Functions  

[logRange](#logRange) : Generate a log spaced range of numbers  
[f2w](#f2w) : Convert from Hz to rad/s     
[w2f](#w2f) : Convert from rad/s to Hz  
[dB](#dB) : Convert from linear gain to dB  
[magPhase](#magPhase) : Convert from complex value to magnitude and phase  
[mag](#mag) : Obtain the magnitude value of a complex number  
[phase](#phase) : Obtain the phase value of a complex number  


<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="8"> Frequency Response Commands </font> 
<BR>
</div>

Commands in this section are useful to measure the response of a circuit against tones of constant frequency. The commands in this section use the slab.py module wave commands to generate and measure the response of the circuit against sine waves.

Note that the **SLab** system is not designed for frequency response characterization. As the hardware boards don't feature any amplification circuit, dynamic range is poor. Using 12 bit ADCs and DACs we can get a maximum dynamic range of 72dB but that's only true at ideal conditions. Taking into account that the signals won't usually reach the full range of the converters and the measurement noise, the dynamic range will be much lower. For instance, with a 10mV noise and signals that only reach half Vdd range, the dynamic range will be reduced to 44dB. That's only two decades for a first order zero or pole.

In practice, using an experimental setup that makes uses of most of the ADC range you can obtain about 60 dB dynamic range in magnitude measurements as dB measurements are logarithmic and thus are quite forgiving for noise errors. Phase measurements, however, will be contaminated with noise much before reaching this point.

You should also take care of low frequency noise as it will limit the available dynamic range.

Frequency response commands use the base transient and wave commands to operate. So, they modify any previous setting configured with **setSampleTime**, **setTransientStorage**, **setWaveFrequency** and erase any wavetable loaded with the any waveform generation command.

<a id='sineGain'></a>

## sineGain

Calculates complex gain for a give frequency.  
Signal is generated at **DAC1** and output is read at one **ADC**.

>**sineGain(v1,v2,freq,channel,npre,maxfs)**  
>Required parameters:  
>$\quad$ v1 : min value of signal  
>$\quad$ v2 : max value of signal  
>$\quad$ freq : frequency (Hz)  
>Optional parameters:    
>$\quad$ channel : ADC channel to read (defaults to 1)  
>$\quad$ npre : number of cycles before measurement (defaults to 5)  
>$\quad$ maxfs : max sample frequency (at least 10*freq)(Defaults to maximum reported by board)  
>Returns complex gain  
 
Obtains the complex gain of a circuit against a sine wave tone. The tone will be applied using **DAC1** and the response of the circuit will be read at the indicated **ADC** channel (1 by default).

![Sine Gain](images/ac/sineGain.png)
 
The sine wave will be generated at **DAC1** as:

$$A_{DAC}=\frac{v_2 - v_1}{2} \qquad M_{DAC}=\frac{v_1+v_2}{2} \qquad DAC_1(t)=M_{DAC} + A_{DAC}sin(2\pi\cdot freq \cdot t)$$
 
So the low peak of the wave will be at **v1** and the high peak of the wave at **v2**.

The output of the circuit read at the ADC is processed to obtain the unknowns $M_{ADC}$, $A_{ADC}$ and $\theta_{ADC}$ that give the best approximation to:

$$ADC(t) \approx M_{ADC} + A_{ADC} sin(2\pi \cdot freq \cdot t + \theta_{ADC})$$
 
After solving for the unknown values, the complex gain is reported as:

$$gain = \frac{A_{ADC}}{A_{DAC}}cos(\theta_{ADC})+j\frac{A_{ADC}}{A_{DAC}}sin(\theta_{ADC})$$

Parameter **npre** selects the number of sine cycles to produce before start measuring the circuit and defaults to 5 cycles.   

Paremeter **maxfs** selets the maximum sample frequency to use and defaults the maximum reported by on the board. If you get a **Sample Overrun** exception it means that measurements are not fast enough for the maximum sample frequency so you will need to select a lower sample frequency. Note that the command will generate an exception if the maximum sample frequency is not, at least, 4 times the tone frequency to use.

At tone frequencies close to the maximum limit, several cycles will be measured to reduce the errors. However, errors will increase the closer you get to the limit, especially in the phase response as any uncalibrated delay between the DAC and the ADC will produce a phase response error.

If the maximum or minimum of the output waveform is too close to the supplies, a saturation warning will be generated.


---

**Example**  
Show the response of a first order low pass filter at the corner frequency.

---

We will test the following circuit.

![RC](images/ac/RC.png)

The corner frequency is:

$$f_c = \frac{1}{2\pi(R1\cdot C1)} \approx 723Hz$$

We can obtain the response of the circuit at this frequency using the following code. Note that we use the default ADC channel 1.

In [ ]:
# EXAMPLE : Response of a RC circuit at the corner frequency
# Theoretical result shall be 0.71 magnitude and -45 deg phase

gain = ac.sineGain(1.0,2.0,723.0)

print('Gain =',gain)
m,p = ac.magPhase(gain)
print('Magnitude =',m)
print('Phase =',p)      

<a id='sineGainAll'></a>

## sineGainAll

Calculates complex gain for a give frequency
Signal is generated at **DAC1** and output is read at all **ADCs**

>**sineGainAll(v1,v2,freq,npre,maxfs)**  
>Required parameters:  
>$\quad$ v1 : min value of signal  
>$\quad$ v2 : max value of signal  
>$\quad$ freq : frequency (Hz)  
>Optional parameters:    
>$\quad$ npre : number of cycles before measurement (defaults to 5)  
>$\quad$ maxfs : max sample frequency (at least 10*freq)(Defaults to maximum reported by board)  
>Returns list of complex gains (one for each ADC)  

Obtains the complex gain of a circuit against a sine wave tone for all **ADCs**. 
The tone will be applied using **DAC1** and the response of the circuit will be read at all **ADC** channels in sequence.
Data is returned as a list of complex gains with one element for each ADC.


---

**Example**  
Show the response of a second order low pass filter at the corner frequency.

---

We will test the following circuit.

![RC](images/ac/RC2.png)

We have two cascade filters with the same corner frequency:

$$f_c = \frac{1}{2\pi(R1\cdot C1)} = \frac{1}{2\pi(R2\cdot C2)} = 1592Hz$$

We can obtain the response of the circuit at the two capacitor nodes.

The code will generate data for all ADCs (8 in the F303 Board), but only ADC1 and ADC2 are usefull.

In [ ]:
# EXAMPLE : Response of a RC circuit at the corner frequency
# Theoretical result shall be 0.71 magnitude and -45 deg phase for ADC1
# Theoretical result shall be 0.5  magnitude and -90 deg phase for ADC1

table = ac.sineGainAll(1.0,2.0,1592.0)

print(table)
print()

m,p = ac.magPhase(table[0])
print('ADC1 Magnitude =',m,'Phase =',p)    
m,p = ac.magPhase(table[1])
print('ADC2 Magnitude =',m,'Phase =',p)

<a id='freqResponse'></a>

## freqResponse

Obtain the frequency response of a circuit
Signal is generated at **DAC1** and output is read at **ADC1**

>**freqResponse(v1,v2,fvector,channel,npre,maxfs)**  
>Required parameters:  
>$\quad$ v1 : min value of signal  
>$\quad$ v2 : max value of signal  
>$\quad$ fvector : vector of frequencies to test  
>Optional parameters:    
>$\quad$ channel : Channel to measure (defaults to 1)  
>$\quad$ npre : number of cycles before measurement (defaults to 5)  
>$\quad$ maxfs : max sample frequency (at least 10*freq)(Defaults to maximum reported by board)  
>Returns a vector of complex gains  

This command computes the complex gain of a circuit by calling the **sineGain** command for the list of frequencies contained in the **fvector** parameter. The command **bodeResponse** issues this command to get the plot data.

That way, the bodeResponse command:

> slab.bodeResponse(0.5,2.5,10.0,8000.0,10,npre=5)

can be implemented as:

> fv = logRange(10.0,8000.0,ppd=10)  
> gv = freqResponse(0.5,2.5,fv,npre=5)  
> plotBode(fv,gv)  

---

**Example**  
Compare the ideal and real response of a first order low pass filter

---

We will test the following circuit.

![RC-10k](images/ac/RC-10k.png)

Then we use the following code to compare the real and ideal response.  
As you can see you get some measurement errors, specially in the phase, at the higher frequencies.

In [ ]:
# EXAMPLE : Compare theoretical and measured response of a RC filter

fv = ac.logRange(10.0,2000.0,10.0)

R = 10000.0
C = 220.0e-9
fc = ac.w2f(1/(R*C)) 

g0 = 1.0/(1.0+1j*fv/fc)
g = ac.freqResponse(0.5,2.5,fv,npre=5,maxfs=35000)

ac.plotBode([fv,fv],[g0,g],["Ideal","Real"])


<a id='freqResponseAll'></a>

## freqResponseAll

Obtain the frequency response of a circuit for all channels
Signal is generated at **DAC1** and output is read at all **ADCs**

>**freqResponseAll(v1,v2,fvector,npre,maxfs)**  
>Required parameters:  
>$\quad$ v1 : min value of signal  
>$\quad$ v2 : max value of signal  
>$\quad$ fvector : vector of frequencies to test  
>Optional parameters:    
>$\quad$ npre : number of cycles before measurement (defaults to 5)  
>$\quad$ maxfs : max sample frequency (at least 10*freq)(Defaults to maximum reported by board)  
>Returns a list of vectors of complex gains  

This command computes the complex gain of a circuit at all ADCs by calling the **sineGain** command for the list of frequencies contained in the **fvector** parameter. 
So it is similar to **freqResponse** but measurements are performed at all ADCs instead of only using ADC1.
Data is returned as a list of complex gains with one element for each ADC.


<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="8"> Frequency Plot Commands </font> 
<BR>
</div>

Commands in this section draw plots of magnitude and phase against a horizontal frequency axis.

<a id='plotBode'></a>

## plotBode

Draws a bode plot
It uses a logarithmic frequency axe
By default it uses dB for the magnitude

>**plotBode(f,g,labels,linear)**  
>Required parameters  
>$\quad$ f : Frequency vector or list of vectors (Hz)  
>$\quad$ g : Gain vector or list of vectors (Complex)  
>Optional parameters:  
>$\quad$ labels : Labels for each curve  
>$\quad$ linear : Use linear vertical axis instead of dB  
>Returns nothing  

Generates a bode plot from frequencies and complex gains.
Frequency horizontal axis will feature logarithmic spacing.

This function can be called in two modes:

In the first mode f and g are lists or numpy arrays that contain frequencies and complex gains. The command will show a bode plot for this data.

In the second mode both f and g are lists of several frequency and gain vectors. The command will superpose several bode plots in this case.

If we use **linear = True** , magnitudes will show in linear values instead of **dB** so the plot will no longer be “Bode” but “Frequency Response”.


In [ ]:
# EXAMPLE : Draw a bode plot of a theoretical 100Hz low pass filter

freq = ac.logRange(10.0,10000.0)
g1 = 1.0/(1.0+1j*freq/100.0)
ac.plotBode(freq,g1)

In [ ]:
# EXAMPLE : Draw a theoretical bode plot of two filters
# 100Hz low pass filter
#  1kHz low pass filter

freq = ac.logRange(10.0,10000.0)
g1 = 1.0/(1.0+1j*freq/100.0)
g2 = 1.0/(1.0+1j*freq/1000.0)
ac.plotBode([freq,freq],[g1,g2],["1k","10k"])

<a id='plotFreq'></a>

## plotFreq

Draws a frequency plot using a linear frequency axis

>**plotFreq(f,v,labels)**  
>Required parameters  
>$\quad$ f : Frequency vector or list of vectors (Hz)  
>$\quad$ v : Complex vector or list of vectors  
>Optional parameters:  
>$\quad$ labels : Labels for each curve  
>Returns nothing  

This command is similar to the **plotBode** command but the spacing in the frequency horizontal axis will be linear instead of logarithmic.
Data in the vertical Y axis will always be linear also.

In [ ]:
# EXAMPLE : Draw a linear frequency response plot of two filters
# 100Hz low pass filter
#  1kHz low pass filter

freq = ac.logRange(10.0,10000.0)
g1 = 1.0/(1.0+1j*freq/100.0)
g2 = 1.0/(1.0+1j*freq/1000.0)
ac.plotFreq([freq,freq],[g1,g2],["1k","10k"])

<a id='bodeResponse'></a>

## bodeResponse

Measures and draws a bode plot

>**bodeResponse(v1,v2,fmin,fmax,ppd,channel,npre,maxfs,returnData)**   
>Required parameters:  
>$\quad$ v1 : min value of signal  
>$\quad$ v2 : max value of signal  
>$\quad$ fmin : minimum frequency  
>$\quad$ fmax : maximum frequency  
>Optional parameters:    
>$\quad$ ppd : number of points per decade (defaults to 10)  
>$\quad$ channel : ADC channel to use (defaults to 1)  
>$\quad$ npre : number of cycles before measurement (defaults to 5)  
>$\quad$ maxfs : max sample frequency (at least 10*freq)(Defaults to maximum reported by board)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData):  
>$\quad$ Tuple of two elements:  
>$\quad$ Frequencies vector  
>$\quad$ Complex gains vector  

This command makes successive calls to **sineGain** to obtain the frequency response of a circuit. After all measurements, results are shown on a bode plot.

Parameteres **v1**, **v2**, **npre** and **maxfs** are the same than in the **sineGain** command. Refer to this command for a deeper explanation.  
Frequencies are spaced geometricaly (linear in logarithmic space). Parameter **fmin** set the minimum frequency to measure, **fmax** the maximum and **ppd** the number of frequencies to test at each decade. If **ppd** is omitted it defaults to 10.

The command returns a tuple of two vectors with the frequency values and the complex gains for the circuit.

>$\quad$fvector, gvector = bodeResponse( .... )

Phase measurement is very sensitive to timing errors. Errors in the phase response can be high for frequencies over 1/20 of the maximum sample frequency of the board if signal level is low. 
In the same way, high attenuation values can get outside of the measurement limits for the SLab system and give wrong results. Special care should be taken for very low frequencies as there could be high levels of low frequency noise,

If optional parameter **returnData** is True or **setPlotReturnData(True)** was called previously, this command returns a tuple with two numpy arrays: frequencies and complex gains.


---

**Example**  
Show the bode response of a first order low pass filter

---

We will test the following circuit.

![RC-10k](images/ac/RC-10k.png)

To obtain the response just execute the following code cell after mounting the circuit.  
You can see that we will have measurement errors, specially in the phase, at the higher frequencies.  

In [ ]:
# EXAMPLE : Bode Plot of a first order low pass filter

ac.bodeResponse(0.5,2.5,10.0,8000.0,10,npre=5)

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="8"> Utility Functions </font> 
<BR>
</div>

Functions in this section take a **float** a **complex** value or a **numpy** array and return an object of the same kind with the same size. For floats and complex values, the function is executed once. For numpy arrays, the function is executed for each element of the array returning an array with the same number of elements than the input array.
As these commands don't use the hardware board, they are all **auxiliary** so you don't need to connect to use them.

<a id='logRange'></a>

## logRange

Generates a logarithmic spaced range of values
    
>**logRange(start,end,ndec,ppd)**  
>Parameters:  
>$\quad$ start : start value  
>$\quad$ end : end value  
>$\quad$ ndec : number of decades  
>Optional parameters:    
>$\quad$ ppd : points per decade  (defaults to 10)  
>Either the end or the ndec parameters must be supplied      
>Returns a vector or values     

Creates a geometrically spaced (linear log spaced) numpy array that starts at the given value. The end of the space can be indicated with the **end** parameter or the **ndec** parameter.
The **ppd** parameter indicates the number of points per decade which defaults to 10.

In [ ]:
# EXAMPLES

fstart =  10
fend   = 100

f = ac.logRange(fstart,fend)           # Range with default 10 ppd
print(f)
print()

f = ac.logRange(fstart,fend,ppd=20)   # Range with 20 ppd
print(f)
print()

f = ac.logRange(fstart,ndec=2)        # 2 decades from fstart with default 10 ppd
print(f)
print()

f = ac.logRange(fstart,ndec=2,ppd=5)   # 2 decades with custom ppd
print(f)

<a id='f2w'></a>

## f2w

Converts frequency from Hz to rad/s  
Returns frequency in rad/s     

In [ ]:
# EXAMPLE

f = 1000
w = ac.f2w(f)
print(f,'Hz equals',w,'rad/s')

<a id='w2f'></a>

## w2f

Converts frequency from rad/s to Hz  
Returns frequency in Hz 

In [ ]:
# EXAMPLE

w = 6283
f = ac.w2f(w)
print(w,'rad/s equals',f,'Hz')

<a id='dB'></a>

## dB

Converts gain from linear to dB values  
Formula applied is:  

$\qquad dB(value) = 20 \cdot log_{10}(value)$

In [ ]:
# EXAMPLE

g = 20
g_dB = ac.dB(g)
print('Linear gain of',g,'equals',g_dB,'dB')

<a id='magPhase'></a>

## magPhase

Take one complex value or a numpy array of complex values and generates two floats or arrays that include the magnitude and phase of each input element. 
Phase is given in degrees (0 to 360).  

In [ ]:
# EXAMPLE

value = 1 + 1j
magnitude,phase = ac.magPhase(value)
print('Complex value',value,'has',magnitude,'magnitude and',phase,'deg phase')

<a id='mag'></a>

## mag

Take one complex or real value or a numpy array of values and generates one float or one array that include the magnitude of each input element. 

In [ ]:
# EXAMPLE

value = 1 + 1j
magnitude = ac.mag(value)
print('Complex value',value,'has',magnitude,'magnitude')

<a id='phase'></a>

## phase

Take one complex value or a numpy array of values and generates one float or one array that include the phase value, in degrees, of each input element. 

In [ ]:
# EXAMPLE

value = 1 + 1j
phase = ac.phase(value)
print('Complex value',value,'has',phase,'deg phase')

## Document license

Copyright  ©  Vicente Jiménez (2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">